# Perceptron, XOR(exclusive OR)

In [1]:
# need library
import numpy as np
import tensorflow as tf
# warning remove
if type(tf.contrib) != type(tf):
    tf.contrib._warning = None
# 딥러닝 구동에 필요한 케라스 함수
from keras.models import Sequential
from keras.layers import Dense   

Using TensorFlow backend.


## XOR problem

In [5]:
import numpy as np

# weight and bias
w11 = np.array([-2,-2])
w12 = np.array([2,2])
w2 = np.array([1,1])
b1 = 3
b2 = -1
b3 = -1

# perceptron function
def MLP(x, w, b):
    y = np.sum(w * x) + b
    if y <= 0:
        return 0
    else:
        return 1
    
# NAND(not AND) gate
def NAND(x1, x2):
    return MLP(np.array([x1, x2]), w11, b1)
# OR gate : 둘 중 하나라도 1이면 1, 그렇지 않으면 0
def OR(x1, x2):
    return MLP(np.array([x1, x2]), w12, b2)
# AND gate : 두가지 모두 1이면 1, 그렇지 않으면 0
def AND(x1, x2):
    return MLP(np.array([x1, x2]), w2, b3)
# XOR gate
def XOR(x1, x2):
    return AND(NAND(x1, x2), OR(x1, x2))

if __name__=='__main__':
    for x in [(0,0),(1,0),(0,1),(1,1)]:
        y = XOR(x[0], x[1])
        print("입력 값: " + str(x) + "출력 값: " + str(y))

입력 값: (0, 0)출력 값: 0
입력 값: (1, 0)출력 값: 1
입력 값: (0, 1)출력 값: 1
입력 값: (1, 1)출력 값: 0


## XOR Back Propogation

In [8]:
import numpy as np
import random

# input and target
data = [
    [[0,0],[0]], [[0,1],[0]],
    [[1,0],[0]], [[1,1],[1]]
]

# 실행 횟수, 학습률, 모멘텀 계수
iterations = 5000
lr = 0.1
mo = 0.9

# 활성화 함수 - 1, sigmoid
# 미분할 때와 아닐 때의 각각의 값
def sigmoid(x, derivative = False):
    if derivative == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

# 활성화 함수 - 2, hyperbolic tangent
# 미분할 때와 아닐 때의 각각의 값
def sigmoid(x, derivative = False):
    if derivative == True:
        return 1 - x ** 2
    return np.tanh(x)

# 가중치 배열을 만드는 함수
def makeMatrix(i, j, fill = 0.0):
    mat = []
    for i in range(i):
        mat.append([fill] * j)
        
# 신경망 실행
class NeuralNetwork:
    
    # 초깃값 지정
    def __init__(self, num_x, num_yh, num_yo, bias = 1):
        self.num_x = num_x + bias # 입력값 , 편향은 1로 지정
        self.num_yh = num_yh # 은닉층의 초깃값
        self.num_yo = num_yo # 출력층의 초깃값
        
        # 활성화 함수 초깃값
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_yo
    
        # 가중치 입력 초깃값
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()
            
        # 가중치 출력 초깃값
        self.weight_out = makeMatrix(self.num_yh, self.num_yo)
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                self.weight_out[j][k] = random.random()
            
        # momentum stochastic gradient descent를 위한 이전 가중치 초깃값
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_yo)
    
    # update function
    def update(self, inputs):
    
        # 입력층의 활성화 함수
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]
        
        # 은닉층의 활성화 함수
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum = sum + self.activation_input[i] * self.weight_in[i][j]
            # sigmoid와 hyperbolic tangent 중 활성화 함수 선택
            self.activation_hidden[j] = tanh(sum, False)
            
        # 출력층의 활성화 함수
        for k in range(self.num_yo):
            sum = 0.0
            for j in range(self.num_yh):
                sum = sum + self.activation_hidden[j] * self.weight_out[j][k]
            # sigmoid와 hyperbolic tangent 중 활성화 함수 선택
            self.activation_out[j] = tanh(sum, False)
            
        return self.activation_out[:]
    
    # 역전파 실행
    def backPropogate(self, targets):
        
        # delta 출력 계산
        output_deltas = [0.0] * self.num_yo
        for k in range(self.num_yo):
            error = targets[k] - self.activation_out[k]
            # sigmoid와 hyperbolic tangent 중 활성화 함수 선택, 미분 적용
            output_deltas[k] = tanh(self.activation_out[k], True) * error
            
        # 은닉 노드와 오차 함수
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_yo):
                error = error + output_deltas[k] * self.weight_out[j][k]
            # sigmoid와 hyperbolic tangent 중 활성화 함수 선택, 미분 적용
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error
            
        # 출력 가중치 업데이트
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo * self.gradient_in[j][k] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient
                
        # 오차 계산(최소제곱법)
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.activation_out[k]) ** 2
        return error
    
    # 학습 실행
    def train(self, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropogate(targets)
            if i % 500 == 0:
                print('error: %-.5f' % error)
                
    # 결과 출력
    def result(self, patterns):
        for p in patterns:
            print('Input: %s, Predict: %s' % (p[0], self.update(p[0])))
            
if __name__ == '__main__':
        
    # 두개의 입력값, 두개의 레이어, 하나의 출력값을 갖도록 설정
    n = NeuralNetwork(2, 2, 1)
    # 학습실행
    n.train(data)
    # 결과출력
    n.result(data)
        
# Reference: http://arctrix.com/nas/python/bpnn.py (Neil Schemenauer)        

TypeError: 'NoneType' object is not subscriptable